# ML Analysis - Система автоматической оценки экзаменов по русскому языку



Заказчик - компания Sigma поставила задачу разработать модель для автоматичексой оценки ответов иностранных граждан при сдаче экзаммена на знание русского языка.

Этот notebook объединяет функциональность обработки CSV файлов и машинного обучения для анализа и экспериментов.

Этот файл:

* принимает на вход тренировочный датасет с данными;

* запрашивает и записывает размер каждого аудиофайла с экзамена;

* производит предобработку и очистку полученных данных;

* анализирует ответ экзаменуемого и выичисляет новые признаки (колонки) и добавляет их в таблицу для будущих предсказаний оценки;

* удаляет лишние ненужные колонки;

* формирует пайплайны для обучения модели;

* выбирает лучшую модель по сетке;

* метрики качества обучения модели представлены в конце файла;

* обученная модель сохраняется в файл.

код включает три разных файла в компиляции. Для бэкенда веб-приложения, этот код из этого файла должен быть разнесен по трем разным файлам.

* backend\handlers\csv_handler.py

* backend\services\ml_service.py

* ml_models\training\train_model.py

Это рабочий файл создан для:

* Исследовательского анализа данных

* Экспериментов с моделями

* Визуализации результатов

* Отладки алгоритмов

## Импорты и настройки

In [107]:
# !pip install mutagen

In [108]:
import pandas as pd
import numpy as np
import pickle
import re
import io
import os
from typing import List, Dict, Any
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict, StratifiedKFold
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import warnings
import pymorphy3
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, RobustScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer 
from phik import phik_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from mutagen.mp3 import MP3
from collections import Counter


In [ ]:
warnings.filterwarnings('ignore') # чтобы не было красный полей с предупреждениями об устаревших библиотеках

%matplotlib inline
plt.ion() # принудительное отображение графиков matplotlib в VS Code
pd.set_option('display.max_colwidth', None) # чтобы df колонки были пошире
pd.set_option('display.float_format', '{:.2f}'.format) # округление чисел в df, чтобы числа не печатал экспоненциально
pd.options.display.expand_frame_repr = False # для принта чтобы колонки не переносил рабоатет тольок в vs code, in jupyter notebook получается каша

In [110]:
path = r'C:\Users\dmi-a\OneDrive\Рабочий стол\DS учеба\Выполненные проекты\rus_exam_site\docs\Данные для кейса.csv'

df = pd.read_csv(path, delimiter=';', encoding='utf-8')

In [111]:
class AudioProcessor(BaseEstimator, TransformerMixin):
    """Отдельный класс для обработки аудиофайлов"""
    
    def __init__(self, timeout=30):
        self.timeout = timeout
        self.counter = 0
        self.total_files = 0
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        self.counter = 0  # Сброс счетчика
        self.total_files = len(X_transformed)
        
        
        if 'Ссылка на оригинальный файл запис' in X_transformed.columns:
            print("Начинаем скачивать аудиофайлы...")
            X_transformed['Длина_файла'] = X_transformed['Ссылка на оригинальный файл запис'].apply(
                self._get_audio_duration
            )
        else:
            X_transformed['Длина_файла'] = 0
            
        return X_transformed
    
    def _get_audio_duration(self, url):
        """Получение длительности аудиофайла"""
        self.counter += 1

        try:
            response = requests.get(url, timeout=self.timeout)
            response.raise_for_status()
            audio_file = MP3(io.BytesIO(response.content))
            duration = audio_file.info.length if audio_file.info.length else 0
            print(f"{self.counter}/{self.total_files} Длина файла {duration / 60:.2f} мин")
            return duration
        except Exception as e:
            print(f"{self.counter}/{self.total_files} Ошибка загрузки аудио: {type(e).__name__}")
            return 0


In [112]:
class DataPreprocessor(BaseEstimator, TransformerMixin):
    """Основной препроцессор данных"""
    
    def __init__(self):
        self.morph = None
        self.fitted = False

    
    def fit(self, X, y=None):
        if self.morph is None:
            self.morph = pymorphy3.MorphAnalyzer()
        self.fitted = True
        return self
    
    def transform(self, X):
        if not self.fitted:
            raise ValueError("Transformer must be fitted before transform")
        
        X_transformed = X.copy()
        original_indices = X_transformed.index.copy()  # Сохраняем исходные индексы

        # Очистка текста
        print('Очистка текста')
        X_transformed = self._clean_text_columns(X_transformed)
        
        # Фильтрация пустых ответов
        print('Фильтрация пустых ответов')
        # Вместо удаления - заполняем пустые ответы
        empty_mask = ~X_transformed['Транскрибация ответа'].str.contains(r'[а-яёa-z]', case=False, na=False)
        X_transformed.loc[empty_mask, 'Транскрибация ответа'] = 'пустой ответ'

        
        # Анализ речи
        print('Анализ речи')
        speech_analysis = X_transformed['Транскрибация ответа'].apply(self._analyze_speech_quality)
        
        # Извлечение признаков речи
        print("Создание признаков речи ['Качество_речи', 'Лексическое_разнообразие', 'Словарный_запас', 'noun_ratio', 'verb_ratio', 'adjective_ratio', 'NOUN', 'VERB', 'ADJF']")
        for feature in ['Качество_речи', 'Лексическое_разнообразие', 'Словарный_запас', 
                       'noun_ratio', 'verb_ratio', 'adjective_ratio', 'NOUN', 'VERB', 'ADJF']:
            X_transformed[feature] = speech_analysis.apply(lambda x: x.get(feature, 0))
        
        # Создание дополнительных признаков
        print("Создание дополнительных признаков")
        X_transformed = self._create_advanced_features(X_transformed)
        X_transformed = self._create_interaction_features(X_transformed)
        
        # Удаление ненужных колонок
        # print("Удаление ненужных колонок")
        # columns_to_drop = ['Id экзамена', 'Id вопроса', 'Ссылка на оригинальный файл запис', 
        #                   'Картинка из вопроса']
        # X_transformed = X_transformed.drop(columns=[col for col in columns_to_drop if col in X_transformed.columns])
        
        return X_transformed
    
    def _clean_text_columns(self, df):
        """Очистка текстовых колонок"""
        for col in ['Текст вопроса', 'Транскрибация ответа']:
            if col in df.columns:
                df[col] = df[col].apply(self._clean_text)
        return df
    
    def _clean_text(self, text):
        """Очистка отдельного текста"""
        if pd.isna(text):
            return text
        
        text = str(text)
        text = re.sub(r'<[^>]+>', '', text)  # HTML теги
        text = text.replace('\n', ' ')        # Переносы строк
        text = re.sub(r'\s+', ' ', text)      # Множественные пробелы
        text = text.strip()                   # Пробелы в начале/конце
        
        return text
    
    def _analyze_speech_quality(self, text):
        """Анализ качества речи"""
        if pd.isna(text) or text == '':
            return self._get_default_analysis()
        
        # Очистка и токенизация
        words = re.findall(r'\b\w+\b', str(text).lower())
        
        if not words:
            return self._get_default_analysis()
        
        # Морфологический анализ
        pos_counts = Counter()
        unique_lemmas = set()
        
        for word in words:
            parsed = self.morph.parse(word)[0]
            
            if parsed.tag.POS:
                pos_counts[parsed.tag.POS] += 1
            
            unique_lemmas.add(parsed.normal_form)
        
        total_words = len(words)
        
        # Расчет метрик
        lexical_diversity = len(unique_lemmas) / total_words if total_words > 0 else 0
        noun_ratio = pos_counts.get('NOUN', 0) / total_words
        verb_ratio = pos_counts.get('VERB', 0) / total_words
        adjective_ratio = pos_counts.get('ADJF', 0) / total_words
        
        complexity_score = self._calculate_complexity(pos_counts, total_words)
        
        quality_score = (
            lexical_diversity + verb_ratio + noun_ratio + 
            adjective_ratio + complexity_score
        )
        
        return {
            'Качество_речи': 190 - (quality_score * 100),
            'Лексическое_разнообразие': lexical_diversity,
            'Словарный_запас': len(unique_lemmas),
            'noun_ratio': noun_ratio,
            'verb_ratio': verb_ratio,
            'adjective_ratio': adjective_ratio,
            'NOUN': pos_counts.get('NOUN', 0),
            'VERB': pos_counts.get('VERB', 0),
            'ADJF': pos_counts.get('ADJF', 0),
        }
    
    def _calculate_complexity(self, pos_counts, total_words):
        """Расчет сложности речи"""
        complexity_weights = {
            'ADJF': 1.6, 'ADVB': 1.8, 'GRND': 1.9, 'PREP': 1.2,
            'CONJ': 1.1, 'VERB': 1.4, 'INFN': 1.3, 'PRTF': 1.5, 'PRTS': 1.5
        }
        
        complexity = 0
        for pos, count in pos_counts.items():
            weight = complexity_weights.get(pos, 1.0)
            complexity += (count / total_words) * weight
        
        return min(complexity, 1)
    
    def _create_advanced_features(self, df):
        """Создание продвинутых признаков"""
        # Пересечение вопроса и ответа
        def text_overlap_ratio(row):
            question = str(row['Текст вопроса']).lower()
            answer = str(row['Транскрибация ответа']).lower()
            
            question_words = set(question.split())
            answer_words = set(answer.split())
            
            if len(question_words) == 0:
                return 0
            
            overlap = len(question_words.intersection(answer_words))
            return overlap / len(question_words)
        
        df['question_overlap_ratio'] = df.apply(text_overlap_ratio, axis=1)
        df['Рассуждение'] = (df['question_overlap_ratio'] >= 0.5).astype(int)
        
        # Длина и скорость
        df['Длина_ответа'] = df['Транскрибация ответа'].str.len()
        df['length_ratio'] = df['Длина_ответа'] / df['Длина_ответа'].mean()
        df['length_factor'] = np.where(df['length_ratio'] > 1, 1, -1)
        
        # Скорость речи (только если есть длина файла)
        if 'Длина_файла' in df.columns:
            df['Скорость_речи'] = df['Длина_ответа'] / (df['Длина_файла'] + 1e-6)  # избегаем деления на 0
        else:
            df['Скорость_речи'] = 0
        
        # Уникальные слова и свобода речи
        df['Уникальных_слов'] = df['Транскрибация ответа'].apply(
            lambda x: len(set(str(x).split()))
        )
        df['Свобода_речи'] = df['Уникальных_слов'] / (df['Длина_ответа'] + 1e-6)
        
        # Общий балл качества
        df['quality_score'] = (
            -df['Рассуждение'] * 1.5 +
            df['length_factor'] * 1.5 +
            df['Скорость_речи'] * 1.5
        ) + 5
        
        return df
    
    def _create_interaction_features(self, df):
        """Создание признаков взаимодействия"""
        df['quality_vocab_interaction'] = df['quality_score'] * df['Словарный_запас'] * 2
        df['length_speed_interaction'] = df['Длина_ответа'] * df['Скорость_речи'] * 2
        df['noun_adjf_interaction'] = df['NOUN'] * df['ADJF'] * 2
        df['diversity_unique_interaction'] = df['Лексическое_разнообразие'] * df['Уникальных_слов'] * 2
        
        return df
    
    def _get_default_analysis(self):
        """Значения по умолчанию для пустого текста"""
        return {
            'Качество_речи': 0, 'Лексическое_разнообразие': 0, 'Словарный_запас': 0,
            'noun_ratio': 0, 'verb_ratio': 0, 'adjective_ratio': 0,
            'NOUN': 0, 'VERB': 0, 'ADJF': 0
        }

In [113]:
# Ограничиваем размер файла для предварительного исследования
# df = df.head(200)

In [114]:
df.shape

(9798, 8)

In [115]:
# Новая ячейка для исследовательского анализа
print("=== ПРЕДВАРИТЕЛЬНАЯ ОБРАБОТКА ДЛЯ ИССЛЕДОВАНИЯ ===")

# Создаем копию для исследования
df_research = df.copy()

# Применяем AudioProcessor
print("1. Обработка аудиофайлов...")
audio_processor = AudioProcessor(timeout=30)
df_research = audio_processor.fit_transform(df_research)

# Применяем DataPreprocessor  
print("2. Анализ речи и создание признаков...")
data_preprocessor = DataPreprocessor()
df_research = data_preprocessor.fit_transform(df_research)

print("3. Обработка завершена!")
print(f"Размер обработанного датасета: {df_research.shape}")
print(f"Новые колонки: {[col for col in df_research.columns if col not in df.columns]}")

=== ПРЕДВАРИТЕЛЬНАЯ ОБРАБОТКА ДЛЯ ИССЛЕДОВАНИЯ ===
1. Обработка аудиофайлов...
Начинаем скачивать аудиофайлы...
1/9798 Длина файла 1.32 мин
2/9798 Длина файла 2.08 мин
3/9798 Длина файла 3.48 мин
4/9798 Длина файла 7.23 мин
5/9798 Длина файла 1.98 мин
6/9798 Длина файла 6.17 мин
7/9798 Длина файла 6.00 мин
8/9798 Длина файла 4.23 мин
9/9798 Длина файла 2.57 мин
10/9798 Длина файла 4.47 мин
11/9798 Длина файла 6.21 мин
12/9798 Длина файла 4.38 мин
13/9798 Длина файла 2.08 мин
14/9798 Длина файла 5.85 мин
15/9798 Длина файла 6.35 мин
16/9798 Длина файла 5.27 мин
17/9798 Длина файла 2.55 мин
18/9798 Длина файла 5.05 мин
19/9798 Длина файла 5.74 мин
20/9798 Длина файла 4.64 мин
21/9798 Длина файла 2.00 мин
22/9798 Длина файла 3.60 мин
23/9798 Длина файла 5.35 мин
24/9798 Длина файла 5.36 мин
25/9798 Длина файла 2.40 мин
26/9798 Длина файла 5.95 мин
27/9798 Длина файла 6.46 мин
28/9798 Длина файла 4.92 мин
29/9798 Длина файла 1.58 мин
30/9798 Длина файла 2.90 мин
31/9798 Длина файла 3.93 ми

# **Перезапускаться отсюда**

In [116]:
df_research.sample(1)

,Id экзамена,Id вопроса,№ вопроса,Текст вопроса,Картинка из вопроса,Оценка экзаменатора,Транскрибация ответа,Ссылка на оригинальный файл запис,Длина_файла,Качество_речи,...,length_ratio,length_factor,Скорость_речи,Уникальных_слов,Свобода_речи,quality_score,quality_vocab_interaction,length_speed_interaction,noun_adjf_interaction,diversity_unique_interaction
3381,3398291,31074201,3,"Вопрос устной части экзамена. Начните диалог. Получите нужную Вам информацию. Будьте вежливы. Представьте ситуацию. Вы пришли в кассы железнодорожного вокзала. Вам нужно купить билеты на поезд до Ташкента. Поздоровайтесь. Узнайте у кассира о наличии билетов, их стоимости. Спросите о скидке для детей. Поблагодарите за полученную информацию и оцените работу кассира.",NaN,1,"Хорошо. Следующая ситуация. Вам нужно начать диалог, получить нужную информацию и быть вежливым. Ситуация. Вы пришли в кассу железнодорожного вокзала. Вам нужно купить билеты на поезд до Ташкента. Поздоровайтесь. Узнайте у кассира о наличии билетов, их стоимости. Спросите о скидке для детей. Поблагодарите за полученную информацию и оцените работу кассира. Здравствуйте. Здравствуйте. Подскажите, пожалуйста, меня интересуют билеты до Ташкента. На какое число? На 27 число. На 27, к сожалению, нет. Есть на 28. Ну, хорошо. И какая стоимость на 28? 7 тысяч. Взрослый билет. Есть какие-то скидки для детей? Сколько лет вашим детям? Детям 7 и 10. На них уже распространяется скидка до 12 лет. 50%. Отлично. Спасибо большое за информацию. Хорошо. Пойдемте.",https://storage.yandexcloud.net/odin-exam-files-dataset/38e3f17f-3c1a-4d5c-826f-e11b11591bc4-2.MP3,339.31,-18.77,...,0.56,-1,2.22,92,0.12,5.33,746.03,3342.11,630,112.98


надо вывести фик матрикс и посмотреть важные признаки

In [117]:
# распределим колонки по типам переменных
numeric_features = [
    'ADJF', # 0.58
    'NOUN', # 0.54
    'Уникальных_слов',    #  0.53
    'Словарный_запас', # 0.52
    'Длина_ответа',  # 0.52
    'length_ratio',  # 0.52
    'quality_vocab_interaction', # 0.51
    'diversity_unique_interaction', # 0.51
    'Лексическое_разнообразие', # 0.50
    'length_speed_interaction', # 0.49
    'VERB',  # 0.49
    'quality_score',  # 0.48 
    'noun_adjf_interaction', # 0.47
    'Скорость_речи',   # 0.46
    'Качество_речи',  # 0.45
    'Длина_файла', # 0.40
       ]

categorical_features = ['№ вопроса'] # 0.52

text_features = [
    'Текст вопроса',  # 0.75
    'Транскрибация ответа' # 0.00
    ]

In [118]:
# Группировка по целевому признаку и вычисление средних
grouped_means = df_research.groupby('Оценка экзаменатора')[numeric_features].mean()
print(grouped_means)

                     ADJF  NOUN  Уникальных_слов  Словарный_запас  Длина_ответа  length_ratio  quality_vocab_interaction  diversity_unique_interaction  Лексическое_разнообразие  length_speed_interaction  VERB  quality_score  noun_adjf_interaction  Скорость_речи  Качество_речи  Длина_файла
Оценка экзаменатора                                                                                                                                                                                                                                                                              
0                   10.03 28.63            93.13            71.36        815.24          0.61                    1570.53                        108.20                      0.63                   8645.79 17.23           9.58                 889.70           4.39         -17.32       200.39
1                   15.30 39.53           125.90            93.34       1108.83          0.83                    2441.28          

In [167]:
#пайплайн для масштабирования и кодировки с использованием ColumnTransformer


preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
    
        
        ('text_question', TfidfVectorizer(max_features=1500), 'Текст вопроса'),
        ('text_answer', TfidfVectorizer(max_features=1500), 'Транскрибация ответа'),
        
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ],
    remainder='drop'
)

preprocessor

,transformers,"[('num', ...), ('text_question', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [168]:
preprocessing_pipeline = Pipeline([
    # ('audio_processor', AudioProcessor()),
    # ('data_prep', DataPreprocessor()),
    ('preprocessor', preprocessor)
])  

In [ ]:

X = df_research.drop(['Оценка экзаменатора'], axis=1)
y = df_research['Оценка экзаменатора']


# Теперь X_processed - это numpy array/sparse matrix
X_processed = preprocessing_pipeline.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42, stratify=y
)

разделить датасет на выборки

теперь надо создать пайплайн категорировать и масштабировать признаки

In [ ]:
# Создаем полный Pipeline
ModelPipeline = Pipeline([  # Обрабатывает все остальное
    ('selector', SelectKBest(f_classif, k=1500)), # 400 = 239, 779, 761
    ('scaler', StandardScaler()), 
    ('classifier', RandomForestClassifier())
])

ModelPipeline

,steps,"[('selector', ...), ('scaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,score_func,<function f_c...001C2DCFF4400>
,k,1500
,copy,True
,with_mean,True
,with_std,True
,n_estimators,100
,criterion,'gini'


создать сетку грид search и сделать кросс-валидацию

In [205]:
RANDOM_STATE=42

scalers = [StandardScaler(with_mean=False), MaxAbsScaler()]

# Обновленная сетка параметров для Pipeline
models_grid = {
    'LogisticRegression': {
        'ModelPipeline': ModelPipeline,
        'params': {
            'scaler': scalers,
            'classifier': [LogisticRegression(
                # class_weight='balanced',  # с балансировкой только хуже
                random_state=RANDOM_STATE, 
                max_iter=1000)],
            'classifier__C': [0.1, 0.3, 0.5, 1, 1.5],
            'classifier__solver': ['liblinear'],
            'classifier__penalty': ['l2'],
            'classifier__max_iter': [20, 25, 28, 30, 32]
        }
    },
    
    # 'GradientBoosting': {
    #     'ModelPipeline': ModelPipeline,
    #     'params': {
    #         'scaler': scalers,
    #         'classifier': [GradientBoostingClassifier(random_state=RANDOM_STATE)],
    #         'classifier__n_estimators': [60, 61, 62, 63, 65],
    #         'classifier__learning_rate': [0.03, 0.05, 0.07, 0.1],
    #         'classifier__max_depth': [1, 2, 3, 4, 5]
    #     }
    # }
}

In [215]:
# GridSearchCV с Pipeline
best_models = {}
model_scores = {}

for name, model in models_grid.items():
    grid_search = GridSearchCV(
        model['ModelPipeline'], 
        model['params'],
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), 
        scoring='neg_mean_absolute_error',
        n_jobs=-1
    )

    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)
    
    best_models[name] = grid_search.best_estimator_
    accuracy = accuracy_score(y_test, y_pred)
    model_scores[name] = accuracy
    mae_test = mean_absolute_error(y_test, y_pred)
    
    print(f"{name}:")
    print(f"train best_score {grid_search.best_score_:.3f}")
    print(f"test (MAE): {mae_test:.3f}")
    print(f"test (accuracy): {accuracy:.3f}")
    print(f"f1_score: {f1_score(y_test, y_pred, average='weighted')}")
    print(f"Лучшие параметры: {grid_search.best_params_}")
    print()

LogisticRegression:
train best_score -0.221
test (MAE): 0.223
test (accuracy): 0.785
f1_score: 0.7625404207199133
Лучшие параметры: {'classifier': LogisticRegression(max_iter=1000, random_state=42), 'classifier__C': 0.3, 'classifier__max_iter': 20, 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear', 'scaler': MaxAbsScaler()}



In [207]:
# После выбора лучшей модели
best_model_name = max(model_scores, key=model_scores.get)
final_best_model = best_models[best_model_name]

In [210]:
final_best_model

,steps,"[('selector', ...), ('scaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,score_func,<function f_c...001C2DCFF4400>
,k,1500
,copy,True
,penalty,'l2'
,dual,False
,tol,0.0001
,C,0.3


In [214]:
# Найти лучшую модель по метрике
best_model_name = max(model_scores, key=model_scores.get)
best_model = best_models[best_model_name]

# Сохранить модель
import pickle
with open('evaluate_exam_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)